# Philly data preparation

Weifan jiang, weifanjiang@g.harvard.edu

In [3]:
import datetime
import csv
import os
import json
import random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

## utility functions

In [4]:
# parse string date
def parse_date(date_str):
    if date_str is None or date_str == '' or date_str == 'None':
        return None
    if date_str.endswith("PST") or date_str.endswith("PDT"):
        date_str = date_str[:-4]
    return datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')


# convert time delta object to number of minutes
def timedelta_to_minutes(timedelta):
    minutes = 0.0
    minutes += timedelta.days * 24 * 60
    minutes += timedelta.seconds / 60.0
    minutes += timedelta.microseconds / (60 * 1000)
    return minutes


# convert datetime object to string representation
def datetime_to_str(t):
    return t.strftime('%Y-%m-%d %H:%M:%S')


# adjust current time by several minutes
def change_time_by_min(t, minutes):
    return t + datetime.timedelta(seconds=60*minutes)


# count the number of machines that a job is scheduled on
# returns (cpu count, gpu count)
def count_machines(detail):
    cpu_count = len(detail)
    gpu_count = 0
    for machine in detail:
        gpu_count += len(machine["gpus"])
    return cpu_count, gpu_count


# rounds a datetime object down to the nearest minute
def round_to_nearest_minute(t):
    return t - datetime.timedelta(seconds=t.second, microseconds=t.microsecond)


# get the time interval from lo to hi, centered at given time
def get_time_interval(center, lo, hi):
    dt = round_to_nearest_minute(parse_date(center))
    return [datetime_to_str(change_time_by_min(dt, x)) for x in range(lo, hi + 1)]


# read csv (the format is not compatible with pandas.read_csv)
def philly_read_csv(fpath, max_lines, desc='loading'):
    columns = None
    data_lists = None
    with open(fpath, "r") as fin:
        reader = csv.reader(fin)
        columns = [x.strip() for x in next(reader)]
        data_lists = [list() for _ in columns]
        pbar = tqdm(total=max_lines, desc=desc)
        for row_raw in reader:
            pbar.update(1)

            # handle missing data: replace "NA" with None
            row = [x if x != "NA" else -1 for x in row_raw]

            # special case for file misformat in gpu utilization trace
            if len(row) != len(columns):
                if row[-1] == "":
                    row = row[:-1]
                if len(row) == 18 and len(columns) == 10:
                    row = row[0:2] + row[2:][::2]
                row = row + [None, ] * (len(columns) - len(row))
            
            # remove the time zone
            if columns[0] == 'time':
                row[0] = row[0][:-4]
            
            for idx, element in enumerate(row):
                data_lists[idx].append(element)
        pbar.close()
    data_dict = dict()
    for colname, elements in zip(columns, data_lists):
        data_dict[colname] = elements
    return pd.DataFrame(data=data_dict)

## preparation

In [5]:
# seed
np.random.seed(10)
random.seed(10)


# data location
trace_dir = "philly-traces/trace-data/"
job_log_path = os.path.join(trace_dir, "cluster_job_log")
output_dir = "data/philly"
sampled_jobs_path = os.path.join(output_dir, "sampled_jobs.json")
os.makedirs(output_dir, exist_ok=True)
for name in ['gpu_util', 'cpu_util', 'mem_util']:
    job_data_dir = os.path.join(output_dir, name)
    os.makedirs(job_data_dir, exist_ok=True)


# schema
time_lo = -10
time_hi = 3
output_columns = ["name", "machine_type", "trace", ]
for i in range(time_lo, time_hi + 1):
    output_columns.append(i)

## sample jobs

In [21]:
if not os.path.isfile(sampled_jobs_path):

    min_start_time = "2017-10-03 01:00:00"
    max_start_time = "2017-12-15 17:42:00"

    # read full data
    with open(job_log_path, "r") as fin:
        job_log = json.load(fin)
    

    # only keep the last attempt of jobs
    jobs_single_attempt = list()
    for job in job_log:
        if len(job["attempts"]) > 1:
            job["attempts"] = [job["attempts"][-1], ]
            if "start_time" not in job["attempts"][0]:
                print(job)
                exit(0)
        # select for pass/fail jobs
        if job['status'] in ('Pass', 'Failed'):
            jobs_single_attempt.append(job)

    # jobs with complete runtime properties
    for job in jobs_single_attempt:
        start_time = parse_date(job["attempts"][0]["start_time"])
        end_time = parse_date(job["attempts"][0]["end_time"])
        if start_time is not None and end_time is not None:
            job["runtime_min"] = timedelta_to_minutes(end_time - start_time)
        else:
            job["runtime_min"] = None
    jobs_single_attempt = [x for x in jobs_single_attempt if x['runtime_min'] is not None]
    # filter for jobs that lasted for at list 5 minutes
    jobs_single_attempt = [x for x in jobs_single_attempt if 5 <= x['runtime_min']]
    # try to select jobs scheduled on multiple GPUs
    jobs_single_attempt = [
        x for x in jobs_single_attempt if count_machines(x["attempts"][0]["detail"])[1] > 1
    ]

    # format output
    output_json = list()
    pbar = tqdm(total=len(jobs_single_attempt), desc="extract sampled jobs")
    for job in jobs_single_attempt:
        pbar.update(1)
        output_job = dict()
        for key in ("status", "vc", "jobid", "submitted_time", "user", "runtime_min"):
            output_job[key] = job[key]
        for key in ("start_time", "end_time", "detail"):
            output_job[key] = job["attempts"][0][key]
        
        # filter for time interval with cpu/gpu/mem logs
        if output_job["start_time"] >= min_start_time and output_job["start_time"] <= max_start_time:
            output_json.append(output_job)
    pbar.close()
    with open(sampled_jobs_path, "w") as fout:
        json.dump(output_json, fout, indent=2)


with open(sampled_jobs_path, "r") as fin:
    sampled_jobs = json.load(fin)

total_job_count = len(sampled_jobs)
pass_job_count = len([x for x in sampled_jobs if x['status'] == 'Pass'])
failed_job_count = len([x for x in sampled_jobs if x['status'] == 'Failed'])

print('total job count {}'.format(total_job_count))
print('Pass jobs {}, failed jobs {}'.format(pass_job_count, failed_job_count))

IndexError: list index out of range

## per-job gpu trace collection

In [2]:
gpu_df = philly_read_csv(os.path.join(trace_dir, "cluster_gpu_util"), 44750640, "load gpu utilization traces")
gpu_df.set_index(["time", "machineId"], inplace=True)
gpu_df.sort_index(inplace=True)

NameError: name 'philly_read_csv' is not defined

In [45]:


for job in tqdm.tqdm_notebook(sampled_jobs, desc='collect job gpu utilizations'):
    job_trace_path = os.path.join(output_dir, "gpu_util", "{}.csv".format(job["jobid"]))
    if not os.path.isfile(job_trace_path):
        data_lists = [list() for _ in output_columns]
        selected_times = get_time_interval(job["start_time"], time_lo, time_hi)
        for assignment in job["detail"]:
            ip = assignment["ip"]
            gpus = assignment["gpus"]
            for gpu in gpus:
                data_lists[0].append("{}_{}".format(ip, gpu))
                data_lists[1].append("gpu")
                data_lists[2].append("utilization")

                for idx, stime in enumerate(selected_times):
                    if (stime, ip) in gpu_df.index:
                        val = gpu_df.loc[(stime, ip), "{}_util".format(gpu)]
                        if val is None:
                            data_lists[3 + idx].append(None)
                        else:
                            data_lists[3 + idx].append(float(val))
                    else:
                        data_lists[3 + idx].append(None)
        data_dict = dict()
        for cname, cvalues in zip(output_columns, data_lists):
            data_dict[cname] = cvalues
        out_df = pd.DataFrame(data=data_dict)
        out_df.to_csv(job_trace_path, index=False)

/tmp/ipykernel_1417703/4016130712.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for job in tqdm.tqdm_notebook(sampled_jobs, desc='collect job gpu utilizations'):


collect job gpu utilizations:   0%|          | 0/3966 [00:00<?, ?it/s]

## per-job cpu trace collection

In [21]:
cpu_df = philly_read_csv(os.path.join(trace_dir, "cluster_cpu_util"), 45028260, "load cpu utilization traces")
cpu_df.set_index(["time", "machine_id"], inplace=True)
cpu_df.sort_index(inplace=True)

NameError: name 'trace_dir' is not defined

In [6]:
for job in tqdm.tqdm_notebook(sampled_jobs, desc='collect job cpu utilizations'):
    job_trace_path = os.path.join(output_dir, "cpu_util", "{}.csv".format(job["jobid"]))
    if not os.path.isfile(job_trace_path):
        data_lists = [list() for _ in output_columns]
        selected_times = get_time_interval(job["start_time"], time_lo, time_hi)
        for assignment in job["detail"]:
            ip = assignment["ip"]
            data_lists[0].append(ip)
            data_lists[1].append("cpu")
            data_lists[2].append("utilization")

            for idx, stime in enumerate(selected_times):
                if (stime, ip) in cpu_df.index:
                    val = cpu_df.loc[(stime, ip)].loc[(stime, ip), "cpu_util"]
                    if val is None:
                        data_lists[3 + idx].append(None)
                    else:
                        data_lists[3 + idx].append(val)
                else:
                    data_lists[3 + idx].append(None)
        data_dict = dict()
        for cname, cvalues in zip(output_columns, data_lists):
            data_dict[cname] = cvalues
        out_df = pd.DataFrame(data=data_dict)
        out_df.to_csv(job_trace_path, index=False)

/tmp/ipykernel_11489/699943188.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for job in tqdm.tqdm_notebook(sampled_jobs, desc='collect job cpu utilizations'):


collect job cpu utilizations:   0%|          | 0/3966 [00:00<?, ?it/s]

## per-job memory utilization

In [36]:
mem_df = philly_read_csv(os.path.join(trace_dir, "cluster_mem_util"), 45003060, "load memory utilization traces")
mem_df.set_index(["time", "machine_id"], inplace=True)
mem_df.sort_index(inplace=True)

/tmp/ipykernel_1417703/2520353373.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm.tqdm_notebook(total=max_lines, desc=desc)


load memory utilization traces:   0%|          | 0/45003060 [00:00<?, ?it/s]

In [38]:
for job in tqdm.tqdm_notebook(sampled_jobs, desc='collect job memory utilizations'):
    job_trace_path = os.path.join(output_dir, "mem_util", "{}.csv".format(job["jobid"]))
    if not os.path.isfile(job_trace_path):
        data_lists = [list() for _ in output_columns]
        selected_times = get_time_interval(job["start_time"], time_lo, time_hi)
        for assignment in job["detail"]:
            ip = assignment["ip"]
            data_lists[0].append(ip)
            data_lists[1].append("cpu")
            data_lists[2].append("memory")

            for idx, stime in enumerate(selected_times):
                if (stime, ip) in mem_df.index:
                    mtotal = mem_df.loc[(stime, ip), "mem_total"]
                    mfree = mem_df.loc[(stime, ip), "mem_free"]
                    if mtotal is None or mfree is None:
                        data_lists[3 + idx].append(None)
                    elif mtotal == -1 or mfree == -1:
                        data_lists[3 + idx].append(-1)
                    else:
                        data_lists[3 + idx].append(100 * float(mfree) / float(mtotal))
                else:
                    data_lists[3 + idx].append(None)
        data_dict = dict()
        for cname, cvalues in zip(output_columns, data_lists):
            data_dict[cname] = cvalues
        out_df = pd.DataFrame(data=data_dict)
        out_df.to_csv(job_trace_path, index=False)

/tmp/ipykernel_1417703/2336382235.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for job in tqdm.tqdm_notebook(sampled_jobs, desc='collect job memory utilizations'):


collect job memory utilizations:   0%|          | 0/3966 [00:00<?, ?it/s]

## Test

In [3]:
import data_reduce

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
philly = data_reduce.PhillyDataReduce(sample_frac=0.3)

In [16]:
df = philly.df_dict["gpu"].fillna(0)
grb = df.groupby(by="jobid")

In [19]:
df.head()

,jobid,status,name,machine_type,trace,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3
0,application_1508364558011_41416,Killed,m143_gpu2,gpu,utilization,81.133333,24.75,2.15,0.0,18.333333,36.45,1.4,1.000000,15.466667,24.05,36.3,61.433333,0.616667,23.766667
1,application_1508364558011_41416,Killed,m143_gpu3,gpu,utilization,74.200000,5.25,2.15,0.0,26.000000,55.60,9.8,19.333333,72.666667,63.70,36.3,61.050000,0.000000,37.183333
2,application_1508364558011_41416,Killed,m143_gpu4,gpu,utilization,73.933333,3.75,0.00,0.0,15.000000,46.75,35.0,1.000000,23.533333,38.35,36.3,67.950000,11.100000,37.950000
3,application_1508364558011_41416,Killed,m143_gpu5,gpu,utilization,76.600000,11.25,0.00,0.0,15.000000,51.30,44.1,1.000000,20.600000,33.15,36.3,63.733333,4.316667,22.616667
4,application_1508364558011_43207,Failed,m403_gpu0,gpu,utilization,100.000000,100.00,100.00,100.0,100.000000,100.00,100.0,100.000000,100.000000,100.00,100.0,100.000000,100.000000,100.000000


In [18]:
agg_df = grb.agg('mean')
agg_df[agg_df["-10"] != 0]

,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3
jobid,,,,,,,,,,,,,,
application_1506638472019_10316,33.625000,37.575000,24.725000,9.150000,31.666667,34.000000,30.150000,25.750000,31.766667,34.358333,31.000000,25.150000,38.000000,46.308333
application_1506638472019_10327,85.625000,75.375000,69.489583,87.175000,82.897917,82.975000,76.750000,43.737500,37.933333,34.031250,40.977083,47.500000,43.535417,41.287500
application_1506638472019_10344,75.314583,75.995833,86.750000,43.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
application_1506638472019_10349,58.643750,56.245833,53.075000,52.458333,46.025000,23.162500,4.450000,10.768750,32.231250,59.112500,48.258333,49.531250,50.306250,24.516667
application_1506638472019_10370,28.333333,18.366667,47.666667,33.125000,24.400000,58.158333,57.650000,58.483333,53.958333,54.833333,30.666667,21.600000,24.000000,23.441667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
application_1513025660013_0806,11.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
application_1513025660013_0808,7.328125,7.156250,7.123958,7.062500,7.203125,7.360417,7.247396,7.189583,7.271875,7.218750,7.229167,7.206250,7.176562,7.268750
application_1513025660013_0987,61.991667,65.775000,46.250000,55.637500,43.000000,46.508333,70.208333,45.350000,15.395833,6.458333,3.583333,2.350000,0.541667,8.150000
